In [5]:
pip install h5py

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import torchvision.models as models

# Load pre-trained ResNet-18
model = models.resnet18(pretrained=True)


In [11]:
for param in model.parameters():
    param.requires_grad = False
# Lưu mô hình vào tệp .pt (hoặc .pth)
torch.save(model.state_dict(), 'resnet18_pretrained.pth')

In [14]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models

# Chuẩn bị dữ liệu
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize dữ liệu
])

# Tải và chuẩn bị dữ liệu CIFAR-10
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False, num_workers=2)

# Khởi tạo mô hình ResNet-18
model = models.resnet18(pretrained=True)

# Đóng băng các layer convolutional ban đầu để giữ lại các trọng số đã học
for param in model.parameters():
    param.requires_grad = False

# Thay đổi fully connected layer (fc) ở cuối để phù hợp với CIFAR-10
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)  # Số lượng classes là 10 cho CIFAR-10

# Load và cập nhật lại trọng số của fc layer từ mô hình đã huấn luyện trước
pretrained_dict = torch.load('resnet18_pretrained.pth')
model_dict = model.state_dict()

# Lọc ra các trọng số của fc layer từ pretrained_dict
pretrained_fc_weight = {k: v for k, v in pretrained_dict.items() if k.startswith('fc.')}

# Cập nhật lại trọng số của fc layer trong model_dict
model_dict.update(pretrained_fc_weight)

# Load lại các trọng số đã cập nhật vào mô hình
model.load_state_dict(model_dict)

# Định nghĩa hàm mất mát và bộ tối ưu hóa
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Huấn luyện mô hình
model.train()

num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # Print every 100 mini-batches
            print(f'Epoch [{epoch + 1}/{num_epochs}], Batch [{i + 1}/{len(trainloader)}], Loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Finished Training')

# Lưu mô hình sau khi huấn luyện
torch.save(model.state_dict(), 'resnet18_finetuned.pth')


Files already downloaded and verified
Files already downloaded and verified


RuntimeError: Error(s) in loading state_dict for ResNet:
	size mismatch for fc.weight: copying a param with shape torch.Size([1000, 512]) from checkpoint, the shape in current model is torch.Size([10, 512]).
	size mismatch for fc.bias: copying a param with shape torch.Size([1000]) from checkpoint, the shape in current model is torch.Size([10]).

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Print statistics every epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

print('Finished Training')


In [ ]:
correct = 0
total = 0
model.eval()
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f} %')
